In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA

# Load both train and test
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# Drop unwanted columns
drop_cols = ['id','composition_label_0','composition_label_1','publication_timestamp',
             'lunar_phase','creator_collective','composition_label_2','track_identifier']

df_train.drop(columns=drop_cols, inplace=True)
df_test.drop(columns=drop_cols, inplace=True)

# Separate features and target in train
target = df_train['target']
df_train.drop(columns=['target'], inplace=True)

# Identify numeric and categorical columns from train (important: use train only for this)
numeric_cols = df_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = df_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Split numeric cols by missingness threshold on train data
threshold = int(0.10 * len(df_train))  # 10%
numeric_null_counts = df_train[numeric_cols].isna().sum()

low_null_numeric = numeric_null_counts[numeric_null_counts <= threshold].index.tolist()
high_null_numeric = numeric_null_counts[numeric_null_counts > threshold].index.tolist()

# ---------- Numeric imputation ----------
# Fit KNN imputer on train low-missing numeric, transform both train and test
knn_imputer = KNNImputer(n_neighbors=3)
train_low_num_imputed = pd.DataFrame(
    knn_imputer.fit_transform(df_train[low_null_numeric]),
    columns=low_null_numeric
)
test_low_num_imputed = pd.DataFrame(
    knn_imputer.transform(df_test[low_null_numeric]),
    columns=low_null_numeric
)

# Fit SimpleImputer on train high-missing numeric, transform both train and test
simple_imputer = SimpleImputer(strategy='mean')
train_high_num_imputed = pd.DataFrame(
    simple_imputer.fit_transform(df_train[high_null_numeric]),
    columns=high_null_numeric
)
test_high_num_imputed = pd.DataFrame(
    simple_imputer.transform(df_test[high_null_numeric]),
    columns=high_null_numeric
)

# Combine numeric imputations for train and test
train_num_imputed = pd.concat([train_low_num_imputed, train_high_num_imputed], axis=1)
test_num_imputed = pd.concat([test_low_num_imputed, test_high_num_imputed], axis=1)

# ---------- Categorical imputation ----------
cat_imputer = SimpleImputer(strategy='most_frequent')
train_cat_imputed = pd.DataFrame(
    cat_imputer.fit_transform(df_train[categorical_cols]),
    columns=categorical_cols
)
test_cat_imputed = pd.DataFrame(
    cat_imputer.transform(df_test[categorical_cols]),
    columns=categorical_cols
)

# OneHotEncoding
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
train_cat_encoded = pd.DataFrame(
    encoder.fit_transform(train_cat_imputed),
    columns=encoder.get_feature_names_out(categorical_cols)
)
test_cat_encoded = pd.DataFrame(
    encoder.transform(test_cat_imputed),
    columns=encoder.get_feature_names_out(categorical_cols)
)

# Combine numeric + categorical features for train and test
train_processed = pd.concat([train_num_imputed.reset_index(drop=True), train_cat_encoded.reset_index(drop=True)], axis=1)
test_processed = pd.concat([test_num_imputed.reset_index(drop=True), test_cat_encoded.reset_index(drop=True)], axis=1)

# ---------- Scaling ----------
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_processed)
test_scaled = scaler.transform(test_processed)



# Final DataFrames
df_train_final = pd.DataFrame(train_scaled, columns=[f"PC{i+1}" for i in range(train_scaled.shape[1])])
df_train_final['target'] = target.reset_index(drop=True)

df_test_final = pd.DataFrame(test_scaled, columns=[f"PC{i+1}" for i in range(test_scaled.shape[1])])

# Now df_train_final and df_test_final are ready for modeling!


In [2]:
import optuna
import pandas as pd
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from lightgbm.callback import early_stopping

# -------------------------------
# Step 0: Prepare Final DataFrames
# -------------------------------

df_train_final = pd.DataFrame(train_scaled, columns=[f"PC{i+1}" for i in range(train_scaled.shape[1])])
df_train_final['target'] = target.reset_index(drop=True)
df_test_final = pd.DataFrame(test_scaled, columns=[f"PC{i+1}" for i in range(test_scaled.shape[1])])

# -------------------------------
# Step 1: Split data for evaluation
# -------------------------------
X = df_train_final.drop(columns=['target'])
y = df_train_final['target']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------------------
# Step 2: Optuna objective functions
# -------------------------------
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'tree_method': 'gpu_hist',
        'random_state': 42
    }
    model = XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)
    preds = model.predict(X_val)
    return mean_squared_error(y_val, preds, squared=False)

def objective_lgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'device': 'gpu',
        'random_state': 42
    }
    model = LGBMRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[early_stopping(stopping_rounds=10)])
    preds = model.predict(X_val)
    return mean_squared_error(y_val, preds, squared=False)

def objective_cat(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'bootstrap_type': 'Bernoulli',
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'task_type': 'GPU',
        'random_seed': 42,
        'verbose': 0
    }
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10, verbose=False)
    preds = model.predict(X_val)
    return mean_squared_error(y_val, preds, squared=False)

# -------------------------------
# Step 3: Hyperparameter tuning
# -------------------------------
print("Tuning XGBoost...")
study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=30)
best_xgb = XGBRegressor(**study_xgb.best_params, tree_method='gpu_hist', random_state=42)
best_xgb.fit(X, y)

print("Tuning LightGBM...")
study_lgb = optuna.create_study(direction='minimize')
study_lgb.optimize(objective_lgb, n_trials=30)
best_lgb = LGBMRegressor(**study_lgb.best_params, device='gpu', random_state=42)
best_lgb.fit(X, y)

print("Tuning CatBoost...")
study_cat = optuna.create_study(direction='minimize')
study_cat.optimize(objective_cat, n_trials=30)

cat_params = study_cat.best_params.copy()
cat_params.update({
    'task_type': 'GPU',
    'random_seed': 42,
    'verbose': 0,
    'bootstrap_type': 'Bernoulli'
})

best_cat = CatBoostRegressor(**cat_params)
best_cat.fit(X, y)

# -------------------------------
# Step 4: Ensemble model
# -------------------------------
ensemble = VotingRegressor([
    ('xgb', best_xgb),
    ('lgb', best_lgb),
    ('cat', best_cat)
])
ensemble.fit(X, y)

# -------------------------------
# Step 5: Evaluate ensemble
# -------------------------------
val_preds = ensemble.predict(X_val)
val_rmse = mean_squared_error(y_val, val_preds, squared=False)
val_mape = mean_absolute_percentage_error(y_val, val_preds)
val_r2 = r2_score(y_val, val_preds)

print("\n📊 Ensemble Validation Metrics:")
print(f" - RMSE: {val_rmse:.4f}")
print(f" - MAPE: {val_mape:.4f}")
print(f" - R²:   {val_r2:.4f}")

# -------------------------------
# Step 6: Predict on test set
# -------------------------------
test_preds = ensemble.predict(df_test_final)

# -------------------------------
# Step 7: Save predictions
# -------------------------------
submission = pd.read_csv("sample_submission.csv")
submission['target'] = test_preds
submission.to_csv("ensemble_submission.csv", index=False)


c:\Users\chand\Desktop\projects\tf_project\tfenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-30 19:04:58,401] A new study created in memory with name: no-name-3de2efcd-1bd7-4003-8280-bb7d7c22a13b


Tuning XGBoost...


c:\Users\chand\Desktop\projects\tf_project\tfenv\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\chand\Desktop\projects\tf_project\tfenv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-05-30 19:05:02,660] Trial 0 finished with value: 11.221333444423117 and parameters: {'n_estimators': 227, 'max_depth': 8, 'learning_rate': 0.27964051937949663, 'subsample': 0.8725241601621998, 'colsample_bytree': 0.8595980244611049}. Best is trial 0 with value: 11.221333444423117.
c:\Users\chand\Desktop\projects\tf_project\tfenv\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` 

Tuning LightGBM...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 10149
[LightGBM] [Info] Number of data points in the train set: 49287, number of used features: 62
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4060 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 48 dense feature groups (2.26 MB) transferred to GPU in 0.006445 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 52.070120
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[969]	valid_0's l2: 130.471


c:\Users\chand\Desktop\projects\tf_project\tfenv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-05-30 19:15:31,550] Trial 0 finished with value: 11.422394565769387 and parameters: {'n_estimators': 969, 'max_depth': 11, 'learning_rate': 0.06989358740583426, 'subsample': 0.9361094720100294, 'colsample_bytree': 0.623068174644005}. Best is trial 0 with value: 11.422394565769387.


...

Tuning CatBoost...


c:\Users\chand\Desktop\projects\tf_project\tfenv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-05-30 19:20:31,332] Trial 0 finished with value: 13.845742839466096 and parameters: {'iterations': 646, 'depth': 4, 'learning_rate': 0.15633468881973317, 'subsample': 0.5658000444935128}. Best is trial 0 with value: 13.845742839466096.
c:\Users\chand\Desktop\projects\tf_project\tfenv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-05-30 19:20:41,399] Trial 1 finished with value: 12.073876884515377 and parameters: {'iterations': 478, 'depth': 9, 'learning_rate': 0.056472779327085935, 'subsample': 0.9338

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 10163
[LightGBM] [Info] Number of data points in the train set: 61609, number of used features: 62
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4060 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 48 dense feature groups (2.82 MB) transferred to GPU in 0.006747 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 52.067328


c:\Users\chand\Desktop\projects\tf_project\tfenv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



📊 Ensemble Validation Metrics:
 - RMSE: 2.6421
 - MAPE: 0.1254
 - R²:   0.9850
